# Proyecto 1 - Visión por Computadora
## Ejercicio 2
## Integrantes:

- Javier Alvarado - 21188
- Mario Guerra - 21008
- Emilio Solano - 21212

In [6]:
import cv2
import numpy as np
import networkx as nx
import json
import os
from skimage.morphology import skeletonize
from scipy import ndimage


def cargar_imagen(ruta_imagen):
    """
    Carga una imagen groundtruth y la convierte a binaria (0/1).
    """
    img = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta_imagen}")
    _, img_bin = cv2.threshold(img, 127, 1, cv2.THRESH_BINARY)
    return img_bin

def obtener_esqueleto(img_bin):
    """
    Obtiene el esqueleto de la imagen binaria usando skeletonize de scikit-image.
    Retorna un array binario (0/1).
    """
    esqueleto_bool = skeletonize(img_bin == 1)
    esqueleto_bin = esqueleto_bool.astype(np.uint8)
    return esqueleto_bin

def clasificar_nodo(num_vecinos):
    """
    Clasifica un nodo (píxel en el esqueleto) según su número de vecinos (en 8 direcciones).
    """
    if num_vecinos == 1:
        return "extremo"       # (verde en la figura)
    elif num_vecinos == 2:
        return "intermedio"    # (gris en la figura)
    elif num_vecinos == 3:
        return "bifurcacion"   # (rojo en la figura)
    elif num_vecinos >= 4:
        return "trifurcacion"  # (azul en la figura)
    else:
        # num_vecinos == 0 sucede si algo sale mal, o no es parte del esqueleto.
        return "desconocido"


def encontrar_vecinos_8(y, x, shape):
    """
    Retorna la lista de coordenadas (y2,x2) que son vecinos en 8 direcciones
    de (y,x), dentro de los límites de la imagen.
    """
    (h, w) = shape
    vecinos = []
    for dy in [-1, 0, 1]:
        for dx in [-1, 0, 1]:
            if dy == 0 and dx == 0:
                continue
            ny, n_x = y + dy, x + dx
            if 0 <= ny < h and 0 <= n_x < w:
                vecinos.append((ny, n_x))
    return vecinos

In [7]:


def construir_grafo_con_clasificacion(esqueleto_bin):
    """
    Construye un grafo donde cada píxel del esqueleto es un nodo.
    Clasifica cada nodo en 'extremo', 'bifurcacion', 'trifurcacion' o 'intermedio'.
    Crea una arista (u,v) para cada par de nodos adyacentes (en 8-direcciones) del esqueleto.
    
    Retorna (G, dict_listas) donde:
      - G es un nx.Graph con atributos de cada nodo.
      - dict_listas es un diccionario con las listas separadas de nodos y la lista de aristas.
    """
    # Dimensiones
    h, w = esqueleto_bin.shape
    
    # Paso 1: recolectar coordenadas de todos los píxeles que pertenezcan al esqueleto
    pixeles_esqueleto = np.argwhere(esqueleto_bin == 1)
    
    # Para mapear coordenadas -> id de nodo y viceversa
    coord_to_id = {}
    for i, (y, x) in enumerate(pixeles_esqueleto):
        coord_to_id[(y, x)] = i
    
    # Crear grafo
    G = nx.Graph()
    
    # Paso 2: Contar vecinos de cada píxel y clasificar
    kernel = np.ones((3,3), dtype=np.uint8)
    kernel[1,1] = 0
    vecinos_count = ndimage.convolve(esqueleto_bin, kernel, mode='constant', cval=0)

    for (y, x) in pixeles_esqueleto:
        num_vec = vecinos_count[y, x]
        tipo = clasificar_nodo(num_vec)
        nodo_id = coord_to_id[(y, x)]
        G.add_node(nodo_id, 
                   pos=(int(y), int(x)),
                   tipo=tipo)
    
    # Paso 3: Crear aristas para cada par de píxeles vecinos en 8 direcciones
    for (y, x) in pixeles_esqueleto:
        nodo_id = coord_to_id[(y, x)]
        vecinos_8 = encontrar_vecinos_8(y, x, (h, w))
        for (ny, n_x) in vecinos_8:
            if esqueleto_bin[ny, n_x] == 1:
                vecino_id = coord_to_id[(ny, n_x)]
                if vecino_id > nodo_id:
                    G.add_edge(nodo_id, vecino_id)
    
    # Ahora creamos un diccionario con las listas solicitadas.
    nodos_extremos = []
    nodos_bifurc = []
    nodos_trifurc = []
    nodos_intermedios = []

    for n in G.nodes():
        tipo = G.nodes[n]['tipo']
        (y, x) = G.nodes[n]['pos']
        if tipo == "extremo":
            nodos_extremos.append({"id": n, "fila": y, "columna": x})
        elif tipo == "bifurcacion":
            nodos_bifurc.append({"id": n, "fila": y, "columna": x})
        elif tipo == "trifurcacion":
            nodos_trifurc.append({"id": n, "fila": y, "columna": x})
        elif tipo == "intermedio":
            nodos_intermedios.append({"id": n, "fila": y, "columna": x})
    
    # Listado de aristas (en amarillo en la figura).
    # Cada arista es un par (u,v). Opcionalmente podríamos guardar la lista de
    # píxeles entre medio, pero aquí cada arista sólo conecta píxeles adyacentes.
    aristas = []
    for (u, v) in G.edges():
        aristas.append({"origen": u, "destino": v})

    dict_listas = {
        "nodos_extremos" : nodos_extremos,
        "nodos_bifurcacion": nodos_bifurc,
        "nodos_trifurcacion": nodos_trifurc,
        "nodos_intermedios": nodos_intermedios,
        "aristas": aristas
    }
    
    return G, dict_listas

def guardar_grafo_json(dict_listas, ruta_salida):
    """
    Guarda en un JSON las listas solicitadas:
    - nodos_extremos, nodos_bifurcacion, nodos_trifurcacion, nodos_intermedios
    - aristas
    """
    with open(ruta_salida, 'w') as f:
        json.dump(dict_listas, f, indent=4)
    print(f"Guardado JSON en {ruta_salida}")

def visualizar_grafo_clasificado(esqueleto_bin, G, ruta_salida=None):
    """
    Visualiza el grafo sobre la imagen. Se colorean los nodos según su tipo:
        - Extremo (verde)
        - Bifurcación (rojo)
        - Trifurcación (azul)
        - Intermedio (gris)
    Y las aristas se dibujan en amarillo.
    """
    # Pasar a BGR para dibujar en color
    # Multiplicamos por 255 para que se vea "blanco" el esqueleto original en la visualización.
    img_color = cv2.cvtColor(esqueleto_bin*255, cv2.COLOR_GRAY2BGR)

    # Dibujamos las aristas en amarillo
    for (u, v) in G.edges():
        y1, x1 = G.nodes[u]['pos']
        y2, x2 = G.nodes[v]['pos']
        # Amarillo = (0,255,255) en BGR
        cv2.line(img_color, (x1,y1), (x2,y2), (0,255,255), 1)
    
    # Dibujamos los nodos según su tipo
    for n in G.nodes():
        y, x = G.nodes[n]['pos']
        tipo = G.nodes[n]['tipo']
        if tipo == "extremo":
            color = (0, 255, 0)    # verde
        elif tipo == "bifurcacion":
            color = (0, 0, 255)    # rojo
        elif tipo == "trifurcacion":
            color = (255, 0, 0)    # azul
        else:
            color = (128, 128, 128) # gris
        cv2.circle(img_color, (x, y), 2, color, -1)

    if ruta_salida:
        cv2.imwrite(ruta_salida, img_color)
        print(f"Visualización guardada en {ruta_salida}")
    
    return img_color

def procesar_imagen(ruta_imagen, ruta_salida_json, ruta_salida_vis=None):
    """
    Procesa una imagen y genera el grafo con la clasificación de nodos.
    """
    # 1) Cargar y esqueletizar
    img_bin = cargar_imagen(ruta_imagen)
    esqueleto_bin = obtener_esqueleto(img_bin)
    
    # 2) Construir grafo con clasificación
    G, dict_listas = construir_grafo_con_clasificacion(esqueleto_bin)
    
    # 3) Guardar JSON
    guardar_grafo_json(dict_listas, ruta_salida_json)
    
    # 4) Visualizar (opcional)
    if ruta_salida_vis:
        visualizar_grafo_clasificado(esqueleto_bin, G, ruta_salida_vis)
    
    return G, dict_listas

def procesar_todas_imagenes(directorio_entrada, directorio_salida):
    """
    Procesa todas las imágenes groundtruth en el directorio especificado.
    """
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)

    for i in range(1, 21):
        nombre_archivo = f"{i}_gt.pgm"
        ruta_imagen = os.path.join(directorio_entrada, nombre_archivo)
        
        if os.path.exists(ruta_imagen):
            print(f"Procesando {nombre_archivo}...")
            ruta_salida_json = os.path.join(directorio_salida, f"{i}_grafo.json")
            ruta_salida_vis = os.path.join(directorio_salida, f"{i}_visualizacion.png")
            procesar_imagen(ruta_imagen, ruta_salida_json, ruta_salida_vis)
        else:
            print(f"No se encontró el archivo {ruta_imagen}")


In [8]:
directorio_entrada = "./data/database/"
directorio_salida = "./resultados/"
procesar_todas_imagenes(directorio_entrada, directorio_salida)

Procesando 1_gt.pgm...
Guardado JSON en ./resultados/1_grafo.json
Visualización guardada en ./resultados/1_visualizacion.png
Procesando 2_gt.pgm...
Guardado JSON en ./resultados/2_grafo.json
Visualización guardada en ./resultados/2_visualizacion.png
Procesando 3_gt.pgm...
Guardado JSON en ./resultados/3_grafo.json
Visualización guardada en ./resultados/3_visualizacion.png
Procesando 4_gt.pgm...
Guardado JSON en ./resultados/4_grafo.json
Visualización guardada en ./resultados/4_visualizacion.png
Procesando 5_gt.pgm...
Guardado JSON en ./resultados/5_grafo.json
Visualización guardada en ./resultados/5_visualizacion.png
Procesando 6_gt.pgm...
Guardado JSON en ./resultados/6_grafo.json
Visualización guardada en ./resultados/6_visualizacion.png
Procesando 7_gt.pgm...
Guardado JSON en ./resultados/7_grafo.json
Visualización guardada en ./resultados/7_visualizacion.png
Procesando 8_gt.pgm...
Guardado JSON en ./resultados/8_grafo.json
Visualización guardada en ./resultados/8_visualizacion.png
